## Steps to deploy Scikit-Learn Model

* Load Data
* Process / Prepare the data.
* Upload the processed data to S3.
* Create Sklearn estimator.
* Deploy the trained model.
* Evaluate the deployed model.

In [1]:
%matplotlib inline

import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.datasets import load_boston
import sklearn.model_selection

In [2]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.predictor import csv_serializer

# This is an object that represents the SageMaker session that we are currently operating in. This
# object contains some useful information that we will need to access later such as our region.
sagemaker_session = sagemaker.Session()

# This is an object that represents the IAM role that we are currently assigned. When we construct
# and launch the training job later we will need to tell it what IAM role it should have. Since our
# use case is relatively simple we will simply assign the training job the role we currently have.
role = get_execution_role()

#### Load Data

In [3]:
boston = load_boston()

#### Save data locally before uploading it to S3

In [4]:
X_bos_pd = pd.DataFrame(boston.data, columns=boston.feature_names)
Y_bos_pd = pd.DataFrame(boston.target)

# We split the dataset into 2/3 training and 1/3 testing sets.
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X_bos_pd, Y_bos_pd, test_size=0.33)

# Then we split the training set further into 2/3 training and 1/3 validation sets.
X_train, X_val, Y_train, Y_val = sklearn.model_selection.train_test_split(X_train, Y_train, test_size=0.33)

In [5]:
data_dir = './data/boston'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [6]:
train = pd.concat([Y_train,X_train],axis=1)
train.to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [7]:
prefix='sklearn-boston'
WORK_DIRECTORY = 'data/boston'

train_input = sagemaker_session.upload_data(WORK_DIRECTORY, key_prefix="{}/{}".format(prefix, WORK_DIRECTORY) )

#### Create SageMaker Scikit Estimator 
   To run our Scikit-learn training script on SageMaker, we construct a sagemaker.sklearn.estimator.sklearn estimator, which accepts several constructor arguments:

* entry_point: The path to the Python script SageMaker runs for training and prediction.
* role: Role ARN
* train_instance_type (optional): The type of SageMaker instances for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* sagemaker_session (optional): The session used to train on Sagemaker.
* hyperparameters (optional): A dictionary passed to the train function as hyperparameters.

In [8]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'scikit-script.py'

sklearn = SKLearn(
    entry_point=script_path,
    train_instance_type="ml.c4.xlarge",
    role=role,
    sagemaker_session=sagemaker_session,
    hyperparameters={'max_leaf_nodes': 30,'max_depth':7})

This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.


#### Train the model

In [9]:
sklearn.fit({'train': train_input})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-07-17 06:00:25 Starting - Starting the training job...
2020-07-17 06:00:27 Starting - Launching requested ML instances......
2020-07-17 06:01:31 Starting - Preparing the instances for training...
2020-07-17 06:02:23 Downloading - Downloading input data...
2020-07-17 06:02:44 Training - Downloading the training image..2020-07-17 06:02:57,502 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-07-17 06:02:57,505 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-07-17 06:02:57,515 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-07-17 06:02:57,770 sagemaker-containers INFO     Module scikit-script does not provide a setup.py. 
Generating setup.py
2020-07-17 06:02:57,770 sagemaker-containers INFO     Generating setup.cfg
2020-07-17 06:02:57,770 sagemaker-containers INFO     Generating MANIFEST.in
2020-07-17 06:02:57,770 sagemaker-containers INFO     Installing module with the follow

#### Deploy model

In [ ]:
predictor = sklearn.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


--------------

#### Make predictions on validation data

In [ ]:
val_pred = predictor.predict(X_val.values)

#### Evaluate Predictions

In [ ]:
from sklearn.metrics import mean_squared_error as mse

print(mse(np.log(Y_val),np.log(val_pred))**0.5)

#### Delete endpoint

In [ ]:
sklearn.delete_endpoint()